# Libraries

In [2]:
# https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset
!pip install kaggle
! mkdir ~/.kaggle

! kaggle datasets download clmentbisaillon/fake-and-real-news-dataset


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
import json
import numpy as np
import re
import random
import pandas as pd

import datetime
from datetime import timedelta
from datetime import datetime

from bs4 import BeautifulSoup

from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

# Functions

In [ ]:
def pre_process(text):
  text = re.sub(r"(?:\@|https?\://)\S+", "", text)
  text = ' '.join(x for x in text.split() if x.startswith('http') == False and x.startswith('www') == False)
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'^www?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  # extra step to make sure html tags are completely removed
  clean = re.compile('<.*>|<.*\"')
  text = re.sub(clean, '', text)
  text = re.sub("(pic.*)","", text)
  text = re.sub(r"http\S+","", text)
  text = re.sub(r"nyti\S+","", text)
  text = re.sub(r"lat\S+","", text)
  text = re.sub(r"bos\S+","", text)
  text = re.sub(r"wapo\S+","", text)
  text = re.sub("(twitter.*)","", text)
  text=re.sub("<!--?.*?-->","",text)
  text = re.sub("(dlvr.*)","", text)
  text=text.replace(u'\u200c'," ")

  return text

In [ ]:
def encoding_fixer(text):

    pat1 = r'@[A-Za-z0-9]+'
    pat2 = r'https?://[A-Za-z0-9./]+'
    combined_pat = r'|'.join((pat1, pat2))

    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [ ]:
def fix_time_format(t):
  try:
    return datetime.strptime(t.strip(' \t\r\n'), "%B %d, %Y")
  except:
    try:
      return datetime.strptime(t.strip(' \t\r\n'), "%b %d, %Y")
    except:
      try:
        return datetime.strptime(t.strip(' \t\r\n'), "%d-%b-%y")
      except:
        print("error")